In [ ]:
import os
import requests
from dotenv import load_dotenv
from urllib.parse import unquote
import xml.etree.ElementTree as ET
import pandas as pd
import time
from datetime import datetime


In [ ]:
# API 정보
load_dotenv()

# 서비스 키 가져오기
service_key = os.getenv('SERVICE_KEY')
decoded_key = unquote(service_key)  
url = 'http://apis.data.go.kr/1400000/forestStusService/getfirestatsservice'

# 날짜 설정
start_date = '20160101'  # 2016년 1월 1일부터 시작
end_date = datetime.today().strftime('%Y%m%d')  # 오늘 날짜

# 요청 기본 파라미터
params = {
    'serviceKey': decoded_key,  
    'numOfRows': '1000',  # 한 번에 1000개씩 요청
    'pageNo': '1',  
    'searchStDt': start_date,  
    'searchEdDt': end_date  # 오늘 날짜까지 데이터 수집
}

# 데이터를 저장할 리스트
all_fire_data = []
page = 1  # 첫 페이지부터 시작

while True:
    print(f"📡 요청 중: 페이지 {page}... (기간: {start_date} ~ {end_date})")
    params['pageNo'] = str(page)  # 페이지 번호 업데이트
    
    # API 요청
    response = requests.get(url, params=params)
    response_text = response.content.decode('utf-8')  # UTF-8 디코딩

    # XML 파싱
    root = ET.fromstring(response_text)
    
    # 전체 데이터 개수 확인 (첫 요청 시만 필요)
    if page == 1:
        total_count = root.find(".//totalCount")
        total_count = int(total_count.text) if total_count is not None else 0
        print(f"📊 총 데이터 개수: {total_count}건")
    
    # 데이터 추출
    fire_data = []
    for item in root.findall(".//item"):
        fire_info = {
            "발생 연도": item.find("startyear").text if item.find("startyear") is not None else "",
            "발생 월": item.find("startmonth").text if item.find("startmonth") is not None else "",
            "발생 일": item.find("startday").text if item.find("startday") is not None else "",
            "발생 요일": item.find("startdayofweek").text if item.find("startdayofweek") is not None else "",
            "발생 시간": item.find("starttime").text if item.find("starttime") is not None else "",
            "진화 연도": item.find("endyear").text if item.find("endyear") is not None else "",
            "진화 월": item.find("endmonth").text if item.find("endmonth") is not None else "",
            "진화 일": item.find("endday").text if item.find("endday") is not None else "",
            "진화 시간": item.find("endtime").text if item.find("endtime") is not None else "",
            "피해 면적(ha)": item.find("damagearea").text if item.find("damagearea") is not None else "",
            "화재 원인": item.find("firecause").text if item.find("firecause") is not None else "",
            "발생 시도": item.find("locsi").text if item.find("locsi") is not None else "",
            "발생 시군구": item.find("locgungu").text if item.find("locgungu") is not None else "",
            "발생 읍면": item.find("locmenu").text if item.find("locmenu") is not None else "",
            "발생 동리": item.find("locdong").text if item.find("locdong") is not None else "",
            "발생 지번": item.find("locbunji").text if item.find("locbunji") is not None else "",
        }
        fire_data.append(fire_info)

    # 가져온 데이터가 없으면 종료
    if not fire_data:
        print("모든 데이터를 가져왔습니다.")
        break

    # 리스트에 데이터 추가
    all_fire_data.extend(fire_data)

    # 현재까지 수집한 데이터 개수 확인
    print(f"현재까지 수집한 데이터 개수: {len(all_fire_data)}건")

    # 페이지 증가
    page += 1

    # API 요청 간격을 조절하여 서버 부하 방지 (필요시 조절)
    time.sleep(1)

# DataFrame으로 변환
df = pd.DataFrame(all_fire_data)

# CSV 파일로 저장 (파일이 있으면 추가)
csv_filename = "forest_fire_data_2016_present.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8-sig", mode='a', header=not pd.io.common.file_exists(csv_filename))

print(f"데이터 저장 완료: {csv_filename}")

📡 요청 중: 페이지 1... (기간: 20160101 ~ 20250224)
📊 총 데이터 개수: 4783건
📥 현재까지 수집한 데이터 개수: 1000건
📡 요청 중: 페이지 2... (기간: 20160101 ~ 20250224)
📥 현재까지 수집한 데이터 개수: 2000건
📡 요청 중: 페이지 3... (기간: 20160101 ~ 20250224)
📥 현재까지 수집한 데이터 개수: 3000건
📡 요청 중: 페이지 4... (기간: 20160101 ~ 20250224)
📥 현재까지 수집한 데이터 개수: 4000건
📡 요청 중: 페이지 5... (기간: 20160101 ~ 20250224)
📥 현재까지 수집한 데이터 개수: 4783건
📡 요청 중: 페이지 6... (기간: 20160101 ~ 20250224)
✅ 모든 데이터를 가져왔습니다.
📂 데이터 저장 완료: forest_fire_data_2016_present.csv


In [25]:
df = pd.read_csv('forest_fire_data_2016_present.csv')
df['주소'] = df.apply(lambda row: f"{row['발생 시도']} {row['발생 시군구']if pd.notna(row['발생 시군구']) else ''}", axis=1)
df.head(5)

,발생 연도,발생 월,발생 일,발생 요일,발생 시간,진화 연도,진화 월,진화 일,진화 시간,피해 면적(ha),화재 원인,발생 시도,발생 시군구,발생 읍면,발생 동리,발생 지번,주소
0,2021,6,8,화요일,14:52:00,2021,6,8,17:00:00,0.04,기타,강원,홍천,내촌,화상대,산9-2,강원 홍천
1,2021,6,7,월요일,15:13:00,2021,6,7,17:00:00,0.01,쓰레기소각,강원,강릉,주문진,향호,산126,강원 강릉
2,2021,6,7,월요일,15:13:00,2021,6,7,16:00:00,0.01,기타,강원,강릉,주문진,향호,산128,강원 강릉
3,2021,5,31,월요일,15:56:00,2021,5,31,18:10:00,0.05,입산자실화,전남,보성,율어,금천,670-2,전남 보성
4,2021,5,27,목요일,15:32:00,2021,5,27,18:00:00,0.13,건축물화재비화,경북,고령,개진,직,산133,경북 고령


In [31]:
from geopy.geocoders import Nominatim

geoloc = Nominatim(user_agent = 'South Korea', timeout=None)

for i in df.index:
    add = df.loc[i, '주소']
    geo = geoloc.geocode(add)
    if geo is not None:
        df.loc[i, 'latitude'] = str(geo.latitude)
        df.loc[i, 'longitude'] = str(geo.longitude)
    else:
        df.loc[i, 'latitude'] = None
        df.loc[i, 'longitude'] = None
df.head(30)


,발생 연도,발생 월,발생 일,발생 요일,발생 시간,진화 연도,진화 월,진화 일,진화 시간,피해 면적(ha),화재 원인,발생 시도,발생 시군구,발생 읍면,발생 동리,발생 지번,주소,latitude,longitude
0,2021,6,8,화요일,14:52:00,2021,6,8,17:00:00,0.04,기타,강원,홍천,내촌,화상대,산9-2,강원 홍천,37.707204000000004,127.90786680993409
1,2021,6,7,월요일,15:13:00,2021,6,7,17:00:00,0.01,쓰레기소각,강원,강릉,주문진,향호,산126,강원 강릉,37.76452,128.8993979
2,2021,6,7,월요일,15:13:00,2021,6,7,16:00:00,0.01,기타,강원,강릉,주문진,향호,산128,강원 강릉,37.76452,128.8993979
3,2021,5,31,월요일,15:56:00,2021,5,31,18:10:00,0.05,입산자실화,전남,보성,율어,금천,670-2,전남 보성,34.766994350000004,127.08166754883318
4,2021,5,27,목요일,15:32:00,2021,5,27,18:00:00,0.13,건축물화재비화,경북,고령,개진,직,산133,경북 고령,35.7016478,128.2499304
5,2021,5,14,금요일,11:53:00,2021,5,14,14:00:00,0.10,기타,강원,횡성,안흥,상안,산35,강원 횡성,37.4828074,128.0104835
6,2021,5,13,목요일,17:02:00,2021,5,13,17:33:00,0.01,쓰레기소각,전남,함평,대동,운교,산160,전남 함평,35.0236844,126.5390794
7,2021,5,13,목요일,15:35:00,2021,5,13,20:10:00,1.20,기타,강원,태백,NaN,하사미,산10외1,강원 태백,37.17609425,128.98423114660807
8,2021,5,12,수요일,04:22:00,2021,5,12,09:00:00,0.30,입산자실화,경기,남양주,화도,묵현,산62-1,경기 남양주,37.6359398,127.2165051
9,2021,5,10,월요일,14:45:00,2021,5,10,16:30:00,0.10,기타,전북,부안,진서,석포,산75-2,전북 부안,35.7269465,126.7392489


In [32]:
df.to_csv('forest_fire_data_with_lag_long.csv', index=False, encoding='utf-8-sig')